In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
from pickle import dump

In [2]:
# Read in csv
df = pd.read_csv("LouisvilleCleanFinal.csv").drop(['Unnamed: 0'],axis=1)
df.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed,BinaryOutcome
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER,DENY
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE,TAKE
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER,TAKE
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER,TAKE
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER,DENY


In [3]:
# # use when different features need different preprocessing
# from sklearn.compose import make_column_transformer
# #  encoding of categorical features
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)

# column_trans = make_column_transformer(
#     (OneHotEncoder(), ['PrimaryColor', 'BreedCategory','IntakeStatus', 'PetAgeCategory','Sex', 'TopBreed']),
#     remainder='passthrough')

# y = ohe.fit_transform(df[['BinaryOutcome']])

# from sklearn.ensemble import RandomForestClassifier as rf

# clf = rf()
# clf.fit(X_train, y_train)


In [4]:
df.columns


Index(['PrimaryColor', 'BreedCategory', 'IntakeStatus', 'PetAgeCategory',
       'Sex', 'TopBreed', 'BinaryOutcome'],
      dtype='object')

In [5]:
X = df.drop(['BinaryOutcome'], axis="columns")
y = df[['BinaryOutcome']]

# intakestatus, PetAgeCategory, Sex
X.head()
y.head()

,BinaryOutcome
0,DENY
1,TAKE
2,TAKE
3,TAKE
4,DENY


In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [7]:
y = ohe.fit_transform(y).toarray()
print(y)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [8]:
X = ohe.fit_transform(X).toarray()

print(X[0:10])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
  1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### NN model


In [10]:
import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=52))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [18]:
 # Compile and fit the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
 model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 200)               10600     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_6 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 102       
Total params: 35,852
Trainable params: 35,852
Non-trainable params: 0
_________________________________________________________________


In [ ]:
 model.fit(
    X_train,
    y_train,
    epochs=60,
    shuffle=True,
    verbose = 2
)

Train on 31151 samples
Epoch 1/60
31151/31151 - 3s - loss: 0.5443 - accuracy: 0.7075
Epoch 2/60
31151/31151 - 2s - loss: 0.5326 - accuracy: 0.7146
Epoch 3/60
31151/31151 - 2s - loss: 0.5292 - accuracy: 0.7155
Epoch 4/60
31151/31151 - 2s - loss: 0.5266 - accuracy: 0.7184
Epoch 5/60
31151/31151 - 2s - loss: 0.5240 - accuracy: 0.7210
Epoch 6/60
31151/31151 - 3s - loss: 0.5209 - accuracy: 0.7244
Epoch 7/60
31151/31151 - 3s - loss: 0.5175 - accuracy: 0.7268
Epoch 8/60
31151/31151 - 2s - loss: 0.5140 - accuracy: 0.7286
Epoch 9/60
31151/31151 - 2s - loss: 0.5117 - accuracy: 0.7296
Epoch 10/60
31151/31151 - 2s - loss: 0.5077 - accuracy: 0.7313
Epoch 11/60
31151/31151 - 2s - loss: 0.5041 - accuracy: 0.7327
Epoch 12/60
31151/31151 - 2s - loss: 0.5012 - accuracy: 0.7353
Epoch 13/60
31151/31151 - 2s - loss: 0.4975 - accuracy: 0.7370
Epoch 14/60
31151/31151 - 2s - loss: 0.4934 - accuracy: 0.7403
Epoch 15/60
31151/31151 - 2s - loss: 0.4907 - accuracy: 0.7400
Epoch 16/60
31151/31151 - 2s - loss: 0.48

In [ ]:

# dump(rf_model, open('rf_modelTESTMarc.pkl', 'wb'))
# new_input = [[0, 1, 0, 0, 1, 0]]
# with open('rf_modelTESTMarc.pkl', 'rb') as file:  
#     model = pickle.load(file)
# model.predict(new_input) 

In [ ]:
# pd.read_pickle('rf_modelTEST2.pkl')